In [31]:
import pandas as pd
import numpy as np

In [17]:
import re
from datasets import load_dataset

ds = load_dataset("proj-persona/PersonaHub", "persona", split="train")

DOMAIN_KEYS = [
    "beauty", "cosmetic", "skincare", "makeup", "fashion", "k-beauty", "korean beauty"
]
TEXT_KEYS = [
    "k-beauty", "korean beauty", "beauty influencer", "makeup artist",
    "skincare", "cushion", "foundation", "lip tint", "olive young", "seoul"
]
domain_cols = [c for c in ds.column_names if "domain" in c.lower()]

In [22]:
def match(row):
    # 1) domain columns에서 keyword matching
    domain_text = " ".join(str(row.get(c, "")) for c in domain_cols).lower()
    domain_ok = any(k in domain_text for k in DOMAIN_KEYS)
    
    # 2) persona text에서 keyword matching
    persona_text = str(row.get("persona", "")).lower()
    text_ok = any(k in persona_text for k in TEXT_KEYS)
    
    return domain_ok or text_ok

In [23]:
kbeauty = ds.filter(match)

Filter:  29%|██▉       | 58000/200000 [00:00<00:00, 183244.92 examples/s]

Filter: 100%|██████████| 200000/200000 [00:01<00:00, 155762.18 examples/s]


In [24]:
kbeauty

Dataset({
    features: ['persona'],
    num_rows: 430
})

In [29]:
kbeauty.to_csv("/workspace/k-beauty-persona-list.csv")

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 24.37ba/s]


44806

In [32]:
kbeauty_short = pd.read_csv("k-beauty-persona-list-short.csv")

In [34]:
kbeauty_short

,persona
0,A fashion influencer seeking skincare advice t...
1,A makeup artist experienced in creating a bron...
2,A makeup artist who adds the finishing touches...
3,A classmate who has successfully cleared their...
4,A makeup artist who started as a follower and ...
5,A makeup artist who has been by their side thr...
6,a makeup artist who specializes in eye makeup.
7,A makeup artist skilled in covering acne scars...
8,A makeup artist and beauty enthusiast passiona...
9,A makeup artist with expertise in bridal makeu...


In [36]:
import json

path = "/workspace/persona-hub/gpt4o_kbeauty_synthesis_output.jsonl"

rows = []

with open(path, "r", encoding="utf-8") as f:
    for line_no, line in enumerate(f, 1):
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
            rows.append({
                "line":line_no,
                "input persona": obj.get("input persona", None),
                "synthesized text": obj.get("synthesized text", None)
            })
        except json.JSONDecodeError as e:
            rows.append({
                "line":line_no,
                "input persona": None,
                "synthesized text": None
                
            })

In [37]:
df = pd.DataFrame(rows)

In [39]:
df

,line,input persona,synthesized text
0,1,A fashion influencer seeking skincare advice t...,"안녕하세요, 여러분! 💖 오늘은 겨울쿨톤에 맞는 파운데이션에 대해 이야기해볼게요. ..."
1,2,A makeup artist experienced in creating a bron...,"안녕하세요, 여러분! 오늘은 겨울쿨톤에 맞는 파운데이션에 대해 이야기해볼게요. 요즘..."
2,3,A makeup artist who adds the finishing touches...,"안녕하세요, 여러분! 오늘은 겨울쿨톤에 맞는 파운데이션에 대해 이야기해볼게요. 요즘..."
3,4,A classmate who has successfully cleared their...,"안녕하세요, 여러분! 오늘은 겨울쿨톤에 맞는 파운데이션 추천해드릴게요. 제가 여드름..."
4,5,A makeup artist who started as a follower and ...,"안녕하세요, 여러분! 💖 오늘은 겨울 쿨톤에 딱 맞는 파운데이션에 대해 이야기해볼게..."
5,6,A makeup artist who has been by their side thr...,"안녕하세요, 여러분! 오늘은 겨울쿨톤에 딱 맞는 파운데이션에 대해 이야기해볼게요. ..."
6,7,a makeup artist who specializes in eye makeup.,"안녕하세요, 여러분! 오늘은 겨울쿨톤에 맞는 파운데이션에 대해 이야기해볼게요. 요즘..."
7,8,A makeup artist skilled in covering acne scars...,"안녕하세요, 여러분! 오늘은 겨울쿨톤 분들에게 딱 맞는 파운데이션에 대해 이야기해볼..."
8,9,A makeup artist and beauty enthusiast passiona...,"안녕하세요, 여러분! 오늘은 겨울쿨톤에 맞는 파운데이션에 대해 얘기해볼게요. 제가 ..."
9,10,A makeup artist with expertise in bridal makeu...,"안녕하세요, 여러분! 💖 오늘은 겨울쿨톤에 잘 어울리는 파운데이션에 대해 이야기해볼..."


In [41]:
df.to_csv("/workspace/check.csv", encoding='utf-8-sig')